In [1]:
from azureml.core import Workspace, Experiment

# Initializing the workspace
workspace_name = "quick-starts-ws-127351"
ws = Workspace.get(name=workspace_name)

# Define an experiment
exp = Experiment(workspace=ws, name=workspace_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127351
Azure region: southcentralus
Subscription id: 2552278b-2817-43a7-820e-5a5a53ff9e19
Resource group: aml-quickstarts-127351


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_configuration = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, "udacity-project", compute_configuration)
compute_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--C": uniform(0.1, 1.0)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
shutil.copy('train.py', './training/')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./training",
              entry_script="train.py",
              compute_target="udacity-project")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=10,
                                     policy=policy,
                                     max_concurrent_runs=2)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run = exp.submit(hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [5]:
# See the progress of the run

RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = run.get_best_run_by_primary_metric()
pickle_filename = "outputs/" + best_run.id + ".pkl"
best_run.download_file(pickle_filename)

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=data_url)
ds

{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [9]:
from azureml.train.automl import AutoMLConfig

training_data = x.join(y)
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=3)

In [10]:
# Submit your automl run

automl_run = exp.submit(config=automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_fa7ca356-9b58-47e7-bcdd-0f8c714d3155

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [11]:
# Retrieve and save your best automl model.

automl_best_run, automl_model = automl_run.get_output()
joblib.dump(value=automl_model, filename="automl-model.pkl")

['automl-model.pkl']

In [12]:
# See more details at the Azure Machine Learning Studio
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-127351,AutoML_fa7ca356-9b58-47e7-bcdd-0f8c714d3155_41,,Completed,Link to Azure Machine Learning studio,Link to Documentation
